In [8]:
!pip install transformers Pillow diffusers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 35.6 MB/s eta 0:00:00a 0:00:01


In [1]:
from transformers import pipeline

/home/openai/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
sentiment = pipeline("text-classification", model="oliverguhr/german-sentiment-bert")

In [4]:
sentiment("Das Auto ist gar nicht mal so gut")

[{'label': 'negative', 'score': 0.9986610412597656}]

In [6]:
sentiment(["Dieser Film ist weder witzig noch intelligent", "Das Produkt ist ohne schwächen"])

[{'label': 'negative', 'score': 0.9850800633430481},
 {'label': 'positive', 'score': 0.9774582386016846}]

In [7]:
en_de_translator = pipeline("translation_en_to_de")
en_de_translator("This could make your life easier.")

No model was supplied, defaulted to google-t5/t5-base and revision 686f1db (https://huggingface.co/google-t5/t5-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
/home/openai/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[{'translation_text': 'Das wird Ihr Leben erleichtern.'}]

In [9]:
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")
sequence_to_classify = "Angela Merkel ist eine Politikerin in Deutschland und war Vorsitzende der CDU"
candidate_labels = ["politics", "economy", "entertainment", "environment"]
output = classifier(sequence_to_classify, candidate_labels, multi_label=False)
print(output["labels"])
print(output["scores"])

['politics', 'economy', 'environment', 'entertainment']
[0.9658799171447754, 0.022846776992082596, 0.007333964109420776, 0.003939350135624409]


## Bildverarbeitung

![Bild](https://unsplash.com/photos/9YUsgVnnsTk/download?ixid=M3wxMjA3fDB8MXxhbGx8N3x8fHx8fHx8MTcxNzQyOTYxMnw&force=true&w=640)

In [5]:
from PIL import Image
import requests

image_classifier = pipeline(task="zero-shot-image-classification", model="openai/clip-vit-large-patch14-336")

# load image
url = 'https://unsplash.com/photos/9YUsgVnnsTk/download?ixid=M3wxMjA3fDB8MXxhbGx8N3x8fHx8fHx8MTcxNzQyOTYxMnw&force=true&w=640'
image = Image.open(requests.get(url, stream=True).raw)

# inference
outputs = image_classifier(url, candidate_labels=["Berlin","Dresden", "New York", "Leipzig"])
outputs

[{'score': 0.9848644733428955, 'label': 'Dresden'},
 {'score': 0.014148855581879616, 'label': 'Leipzig'},
 {'score': 0.000986547558568418, 'label': 'Berlin'},
 {'score': 8.302540521754054e-08, 'label': 'New York'}]

In [9]:
from diffusers import AutoPipelineForText2Image
import torch

pipe = AutoPipelineForText2Image.from_pretrained("stabilityai/sdxl-turbo", torch_dtype=torch.float16, variant="fp16")
pipe.to("cuda")

prompt = "A cinematic shot of a baby racoon wearing an intricate italian priest robe."

image = pipe(prompt=prompt, num_inference_steps=1, guidance_scale=0.0).images[0]
image

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
100%|██████████| 1/1 [00:02<00:00,  2.63s/it]


In [ ]:
# translation

# bilder

# audio

https://www.americanrhetoric.com/mp3clipsXE/politicalspeeches/jfkberlinspeechARXE.mp3


## Audioverarbeitung

[Ich bin ein Berliner](https://www.youtube.com/watch?v=hJNM0C-7lPk&t=15s)

In [ ]:
# um dieses Beispiel auszuführen braucht man ffmpeg 
#!sudo apt install ffmpeg

In [33]:

model_id = "openai/whisper-medium"

pipe = pipeline(
    "automatic-speech-recognition",
    model=model_id,
    chunk_length_s=30,
    batch_size=16,
    device = "cuda:0"
)


result = pipe("./data/speech.mp3")

result["text"]


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


" I am proud to come to this city as the guest of your distinguished mayor who has symbolized throughout the world the fighting spirit of West Berlin. And I am proud to visit the Federal Republic with your distinguished Chancellor who for so many years has committed Germany to democracy and freedom and progress and to come here in the company of my fellow American General Clay who has been in this city during its great moments of crisis and will come again if ever needed. Two thousand years ago, two thousand years ago, the proudest boast was Kiwis Romanus Sum. Today in the world of freedom the't understand or say they don't. What is the great issue between the free world and the communist world. Let them come to Berlin. There are some who say that communism is the wave of the future, let them come to Berlin. And there are some who say in Europe and elsewhere we can work with the communists. Let them come to Berlin. And there are even a few who say that it's true that communism is an ev